In [6]:
# Core ROS and robotics libraries
import rospy
import actionlib
import math
import time

# ROS message types
from geometry_msgs.msg import Point, PoseStamped
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from assignment_2_2024.msg import PlanningAction, PlanningGoal

# Interactive widgets
import ipywidgets as widgets
from IPython.display import display

In [7]:
# Initialize ROS node (only run this once)
try:
    rospy.init_node('Robot_interface', anonymous=True)
    rospy.loginfo("ROS node initialized")
except rospy.ROSException:
    rospy.logwarn("ROS node already exists")

# Verify ROS master connection
try:
    rospy.get_master().getPid()
    print("✅ Connected to ROS master")
except:
    print("❌ Failed to connect to ROS master")
    raise

✅ Connected to ROS master


[WARN] [1746710474.692385, 1480.526000]: ROS node already exists


In [8]:
class ActionClient:
    def __init__(self):
        self.client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
        self.connected = False
        self.connect_to_server()
    
    def connect_to_server(self, timeout=10):
        """Wait for action server with visual feedback"""
        print("🔌 Connecting to action server...")
        for i in range(timeout):
            if self.client.wait_for_server(rospy.Duration(1.0)):
                self.connected = True
                print("✅ Connected to action server!")
                return True
            print(f"⏳ Attempt {i+1}/{timeout}...")
        print("❌ Failed to connect to action server")
        return False

# Test connection
action_client = ActionClient()

🔌 Connecting to action server...
✅ Connected to action server!


In [9]:
class RobotStateMonitor:
    def __init__(self):
        self.position = Point()
        self.min_distance = float('inf')
        
        # Setup subscribers
        rospy.Subscriber('/odom', Odometry, self.odom_callback)
        rospy.Subscriber('/scan', LaserScan, self.scan_callback)
        time.sleep(1)  # Allow time for subscriptions to establish
    
    def odom_callback(self, msg):
        """Update current robot position"""
        self.position = msg.pose.pose.position
    
    def scan_callback(self, msg):
        """Update minimum obstacle distance"""
        valid_ranges = [r for r in msg.ranges if not math.isinf(r) and r > 0.1]
        self.min_distance = min(valid_ranges) if valid_ranges else float('inf')

# Initialize monitor
state_monitor = RobotStateMonitor()
print("📊 Robot state monitor ready")

📊 Robot state monitor ready


In [10]:
# Create input widgets
x_input = widgets.FloatText(value=0.0, description="Target X:")
y_input = widgets.FloatText(value=0.0, description="Target Y:")

# Create control buttons
set_btn = widgets.Button(description="Set Goal", button_style='success')
cancel_btn = widgets.Button(description="Cancel Goal", button_style='danger')

# Create status displays
position_label = widgets.Label(value="Position: (0.00, 0.00)")
distance_label = widgets.Label(value="Closest obstacle: ∞")
status_output = widgets.Output()

# Assemble UI layout
display(widgets.VBox([
    widgets.HTML("<h2>Robot Control Interface</h2>"),
    widgets.HBox([x_input, y_input]),
    widgets.HBox([set_btn, cancel_btn]),
    widgets.HTML("<b>Current Status:</b>"),
    position_label,
    distance_label,
    status_output
]))

In [11]:
def set_goal(_):
    if not action_client.connected:
        with status_output:
            print("⚠️ Not connected to action server!")
        return
    
    goal = PlanningGoal()
    goal.target_pose = PoseStamped()
    goal.target_pose.pose.position.x = x_input.value
    goal.target_pose.pose.position.y = y_input.value
    
    with status_output:
        print(f"🎯 New goal set: ({x_input.value}, {y_input.value})")
    action_client.client.send_goal(goal)

def cancel_goal(_):
    action_client.client.cancel_goal()
    with status_output:
        print("⛔ Current goal cancelled")

# Link buttons to functions
set_btn.on_click(set_goal)
cancel_btn.on_click(cancel_goal)